In [3]:
# LCEL을 이용해서 Map Reduce Chain 직접 구현

from langchain.chat_models import ChatOpenAI
from langchain.document_loaders import UnstructuredFileLoader
from langchain.text_splitter import CharacterTextSplitter
from langchain.embeddings import OpenAIEmbeddings, CacheBackedEmbeddings
from langchain.vectorstores import FAISS
from langchain.storage import LocalFileStore
from langchain.prompts import ChatPromptTemplate
from langchain.schema.runnable import RunnablePassthrough, RunnableLambda

# ChatGPT 모델 초기화 - 온도값 0.1로 설정하여 일관된 응답 생성
llm = ChatOpenAI(
    temperature=0.1,
)

# 임베딩 캐시를 저장할 로컬 디렉토리 설정
cache_dir = LocalFileStore("./.cache/")

# 문서 분할기 설정 - tiktoken 토크나이저 사용
# 각 청크는 600자, 100자 오버랩으로 분할
splitter = CharacterTextSplitter.from_tiktoken_encoder(
    separator="\n",
    chunk_size=600,
    chunk_overlap=100,
)

# test.txt 파일 로더 초기화
loader = UnstructuredFileLoader("./files/test.txt")

# 문서를 청크로 분할
docs = loader.load_and_split(text_splitter=splitter)

# OpenAI 임베딩 모델 초기화
embeddings = OpenAIEmbeddings()

# 임베딩 결과를 로컬에 캐시하도록 설정
cached_embeddings = CacheBackedEmbeddings.from_bytes_store(embeddings, cache_dir)

# FAISS 벡터 저장소 생성 - 문서와 임베딩 결과 저장
vectorstore = FAISS.from_documents(docs, cached_embeddings)

# 벡터 저장소를 검색기로 변환
retriever = vectorstore.as_retriever()

# 각 문서 청크를 처리할 프롬프트 템플릿 정의
map_doc_prompt = ChatPromptTemplate.from_messages(
    [
        (
            "system",
            """
            Use the following portion of a long document to see if any of the text is relevant to answer the question. Return any relevant text verbatim. If there is no relevant text, return : ''
            -------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 프롬프트 템플릿과 LLM을 연결하여 각 문서 청크를 처리할 체인 생성
# map_doc_prompt: 문서 청크에서 관련 내용을 추출하는 프롬프트
# llm: ChatGPT 모델
# | 연산자로 프롬프트와 LLM을 연결하여 하나의 실행 가능한 체인으로 만듦
map_doc_chain = map_doc_prompt | llm

# 각 문서에서 질문과 관련된 내용을 추출하여 하나의 문자열로 합치는 함수
# inputs: documents(검색된 문서들), question(사용자 질문)
# return: 각 문서에서 추출된 관련 내용들을 개행으로 구분하여 합친 문자열
def map_docs(inputs):
    documents = inputs["documents"]
    question = inputs["question"]
    return "\n\n".join(
        map_doc_chain.invoke({
            "context": doc.page_content, "question": question
        }).content for doc in documents
    )

# 검색기와 질문을 입력으로 받아 map_docs 함수를 실행하는 체인 생성
# documents: retriever로 검색된 관련 문서들
# question: 사용자의 질문을 그대로 전달
# RunnableLambda(map_docs)로 검색된 문서들에서 질문과 관련된 내용을 추출하여 하나의 문자열로 합침
map_chain = {
    "documents": retriever,
    "question": RunnablePassthrough(),
} | RunnableLambda(map_docs)

# LLM에게 전달될 최종 프롬프트 내용
final_prompt = ChatPromptTemplate.from_messages(
    # 시스템 프롬프트 내용 : 긴 문서와 질문에서 추출된 다음 부분을 바탕으로 최종 답변을 작성하세요. 답을 모르면 모른다고 하면 됩니다. 답을 만들어내려고 하지 마세요.
    [
        (
            "system",
            """
            Given the following extracted parts of a long document and a question, create a final answer. 
            If you don't know the answer, just say that you don't know. Don't try to make up an answer.
            ------
            {context}
            """,
        ),
        ("human", "{question}"),
    ]
)

# 최종 체인 생성
# context: map_chain으로 검색된 문서들에서 추출한 관련 내용
# question: 사용자의 질문을 그대로 전달
# final_prompt와 llm을 연결하여 최종 답변을 생성하는 체인
chain = {"context": map_chain, "question": RunnablePassthrough()} | final_prompt | llm

# chain.invoke()를 사용하여 질문을 처리. "등장인물들은 누구인가요?"라는 질문을 체인에 전달하여 답변을 생성하고 출력
# 1. retriever로 관련 문서 검색
# 2. map_chain으로 문서에서 관련 내용 추출 
# 3. final_prompt와 llm으로 최종 답변 생성
print(chain.invoke("등장인물들은 누구인가요?"))


content='죄송합니다, 제가 그 정보를 알 수 없습니다.'
